Notebook to convert data into panel format for regressions

In [1]:
import pandas as pd

In [2]:
# Load aggregated dataframes
month_agg = pd.read_csv('data/aggregated_data.csv')
week_agg = pd.read_csv('data/aggregated_data_weekly.csv')

In [3]:
month_agg.head()

,Unnamed: 0,year_month,launched,share_with_ukraine,a3_share_with_ukraine,la6_share_with_ukraine,t5_share_with_ukraine,fatalities,a3_war_share_out_of_total,la6_war_share_out_of_total,t5_war_share_out_of_total,combined_war_share_out_of_total,a3_share_class_1,la6_share_class_1,t5_share_class_1,total_share_class_1
0,0,2022-12-01,364.0,0.068872,0.091422,0.059908,0.058140,2174,0.020691,0.013054,0.011542,0.014037,0.178934,0.208306,0.196842,0.196247
1,1,2023-01-01,213.0,0.102060,0.147257,0.079872,0.079861,1187,0.021963,0.025506,0.023688,0.024198,0.254697,0.317797,0.293921,0.296647
2,2,2023-02-01,193.0,0.141184,0.195722,0.100925,0.137572,665,0.035792,0.021755,0.027630,0.025175,0.210837,0.215436,0.199777,0.207990
3,3,2023-03-01,181.0,0.090881,0.111977,0.076399,0.088000,866,0.027419,0.019478,0.024133,0.021417,0.281768,0.254028,0.271830,0.262083
4,4,2023-04-01,117.0,0.072224,0.091097,0.061329,0.065614,1135,0.021889,0.014657,0.018935,0.017272,0.218646,0.251767,0.286108,0.257773


In [4]:
week_agg.head()

,Unnamed: 0,week,launched,share_with_ukraine,a3_share_with_ukraine,la6_share_with_ukraine,t5_share_with_ukraine,fatalities,a3_war_share_out_of_total,la6_war_share_out_of_total,t5_war_share_out_of_total,count_class_one,total_seconds,combined_war_share_out_of_total,a3_share_class_1,la6_share_class_1,t5_share_class_1,total_share_class_1
0,0,2022-12-05,99.0,0.082557,0.082251,0.081761,0.084158,313,0.018939,0.017505,0.012624,562.0,35160,0.015984,0.155602,0.213146,0.148833,0.182645
1,1,2022-12-12,116.0,0.038674,0.048458,0.046429,0.018433,282,0.007407,0.007884,0.004608,190.0,29640,0.006410,0.262295,0.171946,0.246914,0.196484
2,2,2022-12-19,38.0,0.088710,0.115000,0.072581,0.081395,646,0.018333,0.018481,0.016279,663.0,37200,0.017823,0.159074,0.253223,0.198582,0.200000
3,3,2022-12-26,156.0,0.077961,0.131250,0.051351,0.087591,972,0.025850,0.011040,0.025791,570.0,32760,0.017399,0.168701,0.189512,0.292414,0.210100
4,4,2023-01-02,41.0,0.076590,0.074257,0.088123,0.065502,211,0.007284,0.024266,0.012809,615.0,37500,0.016400,0.122661,0.274368,0.193407,0.221542


In [5]:
# Drop cols which are aggregated across channel
month_agg = month_agg.drop(['share_with_ukraine', 'total_share_class_1', 'combined_war_share_out_of_total'], axis=1)
week_agg = week_agg.drop(['share_with_ukraine', 'total_share_class_1', 'combined_war_share_out_of_total'], axis=1)

Convert month df to panel

In [6]:
# Melting the dataframe to long format
df_long = pd.melt(month_agg, id_vars=['year_month', 'launched', 'fatalities'],
                  value_vars=['a3_share_with_ukraine', 'la6_share_with_ukraine', 't5_share_with_ukraine', 
                              'a3_share_class_1', 'la6_share_class_1', 't5_share_class_1',
                              'a3_war_share_out_of_total', 'la6_war_share_out_of_total', 't5_war_share_out_of_total'],
                  var_name='type', value_name='value')

# Extracting the 'a3', 't5', 'la6' part and the share type part
df_long[['share_type', 'type']] = df_long['type'].str.extract(r'(a3|la6|t5)_(.*)')

# Pivoting the dataframe to wide format again
df_panel = df_long.pivot_table(index=['year_month', 'launched', 'fatalities', 'share_type'], 
                               columns='type', values='value').reset_index()

# Reordering the columns
df_panel_month = df_panel[['year_month', 'share_type', 'launched', 
                     'share_with_ukraine', 'fatalities', 'share_class_1', 'war_share_out_of_total']]

# Renaming columns
df_panel_month.columns = ['year_month', 'channel', 'launched', 
                    'coverage', 'fatalities', 'war_images', 'war_share_out_of_total']

In [7]:
df_panel_month

,year_month,channel,launched,coverage,fatalities,war_images,war_share_out_of_total
0,2022-12-01,a3,364.0,0.091422,2174,0.178934,0.020691
1,2022-12-01,la6,364.0,0.059908,2174,0.208306,0.013054
2,2022-12-01,t5,364.0,0.058140,2174,0.196842,0.011542
3,2023-01-01,a3,213.0,0.147257,1187,0.254697,0.021963
4,2023-01-01,la6,213.0,0.079872,1187,0.317797,0.025506
5,2023-01-01,t5,213.0,0.079861,1187,0.293921,0.023688
6,2023-02-01,a3,193.0,0.195722,665,0.210837,0.035792
7,2023-02-01,la6,193.0,0.100925,665,0.215436,0.021755
8,2023-02-01,t5,193.0,0.137572,665,0.199777,0.027630
9,2023-03-01,a3,181.0,0.111977,866,0.281768,0.027419


Convert week df to panel

In [8]:
# Melting the dataframe to long format
df_long = pd.melt(week_agg, id_vars=['week', 'launched', 'fatalities'],
                  value_vars=['a3_share_with_ukraine', 'la6_share_with_ukraine', 't5_share_with_ukraine', 
                              'a3_share_class_1', 'la6_share_class_1', 't5_share_class_1',
                              'a3_war_share_out_of_total', 'la6_war_share_out_of_total', 't5_war_share_out_of_total'],
                  var_name='type', value_name='value')

# Extracting the 'a3', 't5', 'la6' part and the share type part
df_long[['share_type', 'type']] = df_long['type'].str.extract(r'(a3|la6|t5)_(.*)')

# Pivoting the dataframe to wide format again
df_panel = df_long.pivot_table(index=['week', 'launched', 'fatalities', 'share_type'], 
                               columns='type', values='value').reset_index()

# Reordering the columns
df_panel_week = df_panel[['week', 'share_type', 'launched', 
                     'share_with_ukraine', 'fatalities', 'share_class_1', 'war_share_out_of_total']]

# Renaming columns
df_panel_week.columns = ['week', 'channel', 'launched', 
                    'coverage', 'fatalities', 'war_images', 'war_share_out_of_total']

In [9]:
df_panel_week

,week,channel,launched,coverage,fatalities,war_images,war_share_out_of_total
0,2022-12-05,a3,99.0,0.082251,313,0.155602,0.018939
1,2022-12-05,la6,99.0,0.081761,313,0.213146,0.017505
2,2022-12-05,t5,99.0,0.084158,313,0.148833,0.012624
3,2022-12-12,a3,116.0,0.048458,282,0.262295,0.007407
4,2022-12-12,la6,116.0,0.046429,282,0.171946,0.007884
...,...,...,...,...,...,...,...
210,2024-04-22,a3,92.0,0.030973,135,0.204276,0.006342
211,2024-04-22,la6,92.0,0.056818,135,0.281457,0.016098
212,2024-04-22,t5,92.0,0.000000,135,0.327869,0.005435
213,2024-04-29,a3,56.0,0.032258,67,0.267081,0.006183


Save dataframes

In [10]:
df_panel_month.to_csv('data/monthly_panel.csv', index=False)
df_panel_week.to_csv('data/weekly_panel.csv', index=False)